In [2]:
import pandas as pd 
import pyspark

In [12]:
### From official documentation
# Import SparkSession
from pyspark.sql import SparkSession

# Create SparkSession 
spark = SparkSession.builder \
      .master("local[1]") \
      .appName("my_pipeline") \
      .config("spark.sql.repl.eagerEval.enabled", True) \
      .getOrCreate()
print(spark.version)


# Load the dataset
df = spark.read.csv("../ecommerce_data_with_trends.csv", header=True, inferSchema=True)

# Show the first 5 rows
df.show(5)



3.5.1
+--------------------+--------------------+-----------+--------------+-----------------+-------------+--------------------+--------------------+------+--------+------------+
|      transaction_id|           timestamp|customer_id| customer_name|             city|customer_type|        product_name|            category| price|quantity|total_amount|
+--------------------+--------------------+-----------+--------------+-----------------+-------------+--------------------+--------------------+------+--------+------------+
|TX_89a20095-f7be-...|2023-10-30 03:01:...|       6933|    David Hays|      New Sabrina|          B2C|Furniture Product_10|Home & Kitchen > ...|246.08|       4|      984.32|
|TX_a6b15a50-47b9-...|2023-10-30 03:06:...|       9328| Adam Oconnell|East Katherineton|          B2C|Non-Fiction Produ...| Books > Non-Fiction| 792.3|       4|      3169.2|
|TX_abdde2cb-3752-...|2023-10-30 03:06:...|       6766|   Jerry Brown|         Lukefort|          B2B|   Bedding Product_1|H

3. Creating a Temporary View
Once you have your data in a DataFrame, you can create a temporary view to run SQL queries against it. A temporary view is a named view of a DataFrame that is accessible only within the current Spark session.

To create a temporary view, use the createOrReplaceTempView method

df.createOrReplaceTempView("sales_data")

In [4]:
df.createOrReplaceTempView("ecommerce_data")

In [13]:
result = spark.sql("SELECT * FROM ecommerce_data")
result.show(5)

+--------------------+--------------------+-----------+--------------+-----------------+-------------+--------------------+--------------------+------+--------+------------+
|      transaction_id|           timestamp|customer_id| customer_name|             city|customer_type|        product_name|            category| price|quantity|total_amount|
+--------------------+--------------------+-----------+--------------+-----------------+-------------+--------------------+--------------------+------+--------+------------+
|TX_89a20095-f7be-...|2023-10-30 03:01:...|       6933|    David Hays|      New Sabrina|          B2C|Furniture Product_10|Home & Kitchen > ...|246.08|       4|      984.32|
|TX_a6b15a50-47b9-...|2023-10-30 03:06:...|       9328| Adam Oconnell|East Katherineton|          B2C|Non-Fiction Produ...| Books > Non-Fiction| 792.3|       4|      3169.2|
|TX_abdde2cb-3752-...|2023-10-30 03:06:...|       6766|   Jerry Brown|         Lukefort|          B2B|   Bedding Product_1|Home & 

- Finding top customers by total spend. 
- Analyzing purchase frequency by customer segment. 
- Identifying trends in product purchases over time.

### Finding top customers by total spend. 
Simple query that groups by customer_id and sums the total spend. 

In [33]:

query = """
SELECT 
    customer_id,
    customer_name, 
    city,
    customer_type,
    COUNT(*) AS number_of_transactions,
    SUM(total_amount) AS total_orders_sum
FROM ecommerce_data
GROUP BY customer_id, customer_name, city, customer_type
ORDER BY total_orders_sum DESC;

"""

result = spark.sql(query)
result.show(5)

+-----------+------------------+---------------+-------------+----------------------+------------------+
|customer_id|     customer_name|           city|customer_type|number_of_transactions|  total_orders_sum|
+-----------+------------------+---------------+-------------+----------------------+------------------+
|       3970|      Shannon Koch|     Oliviabury|          B2B|                   128|        4241408.29|
|       4417|       Sharon Case|     West Laura|          B2B|                   133|        4167603.02|
|        657|   Melissa Jackson|South Tracyland|          B2B|                   120| 4124347.750000001|
|       9019|      Vanessa Meza|New Paulchester|          B2B|                   127|4084312.5299999993|
|       7625|Caroline Blanchard|   Port Melissa|          B2B|                   123|4047170.2399999984|
+-----------+------------------+---------------+-------------+----------------------+------------------+
only showing top 5 rows



### Analyzing purchase frequency by customer segment. 
Here we will group by customer segment and count the number of purchases.\
It corresponds to answering that question : "how many transactions have they made during one month ?".

So for example if a customer_X has made 3 transactions in January, 2 in February and 4 in March,\
The result will be 
```
customer_X
3 -> January,
2 -> February,
4 -> March
```

If needed, the window of one month can be changed to a week or a year.

In [43]:
query = """
SELECT 
    customer_id,
    customer_name, 
    city,
    customer_type,
    COUNT(*) AS number_of_transactions,
    MONTH(timestamp) AS month,
    YEAR(timestamp) AS year
FROM ecommerce_data
GROUP BY year, month, customer_id, customer_name, city, customer_type
ORDER BY customer_type, customer_id , year, month ASC;
"""

result = spark.sql(query)
print(result.count())
result.show(150)


123951
+-----------+--------------------+-----------------+-------------+----------------------+-----+----+
|customer_id|       customer_name|             city|customer_type|number_of_transactions|month|year|
+-----------+--------------------+-----------------+-------------+----------------------+-----+----+
|          1|    Kimberly Watkins|   Alexanderville|          B2B|                     4|   10|2023|
|          1|    Kimberly Watkins|   Alexanderville|          B2B|                     7|   11|2023|
|          1|    Kimberly Watkins|   Alexanderville|          B2B|                    11|   12|2023|
|          1|    Kimberly Watkins|   Alexanderville|          B2B|                     9|    1|2024|
|          1|    Kimberly Watkins|   Alexanderville|          B2B|                     4|    2|2024|
|          1|    Kimberly Watkins|   Alexanderville|          B2B|                    16|    3|2024|
|          1|    Kimberly Watkins|   Alexanderville|          B2B|                  

### Identifying trends in product purchases over time.
1. Use CTE to create a temporary view of the data and then run SQL queries against it.

In [30]:
query = """
WITH temp_table AS (
    SELECT 
        product_name,
        SUM(total_amount) AS total_amount,
        MONTH(timestamp) AS month,
        YEAR(timestamp) AS year
    FROM ecommerce_data
    GROUP BY product_name, year, month
)

SELECT 
    product_name,
    year,
    month,
    total_amount
FROM temp_table
ORDER BY product_name, year, month ASC
"""

result = spark.sql(query)
count_rows = result.count()
print(count_rows)


4563


2. Item names are ordered using human design sorting order.
ie 
- 1 -> 2 -> 3 -> 4 -> 10 -> 11 -> 12 -> 13 -> 14 
- instead of 
- 1 -> 10 -> 11 -> 12 -> 13 -> 14 -> 2 -> 3 -> 4

In [ ]:
query = """
SELECT 
    product_name,
    SUM(total_amount) AS total_amount,
    MONTH(timestamp) AS month,
    YEAR(timestamp) AS year
FROM ecommerce_data
GROUP BY product_name, year, month
ORDER BY 
        CONCAT( REPEAT(  "0", 18 - LENGTH( product_name ) ) , product_name ) ASC,
        year,
        month ASC
"""

result = spark.sql(query)
count_rows = result.count()
print(count_rows)

result.show(150, truncate=False)